<a href="https://colab.research.google.com/github/mtpradoc/labs/blob/master/Guidelines_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Guidelines for Prompting

In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

##Setup

**Load the API key and relevant Python libraries**

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.4 MB/s eta 0:00:00


In this way, you can load the OpenAI API key for you.

In [15]:
import openai
import os

openai.api_key = OPENAI_API_KEY

**helper function**

We will use OpenAI's gpt-3.5-turbo model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs:

In [7]:
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{'role': "user", "content": prompt}]
  response = openai.ChatCompletion.create(
      model=model,
      messages=messages,
      temperature=0, # this is the degree of randomness of the model's output
  )
  return response.choices[0].message["content"]

##Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to "think"**

###Tactics
**Tactic 1: User delimiters to clearly indicate distinct parts of the input**


*   Delimiters can be anything like: ```, """,<>,<tag> </tag>,:


In [16]:
text = f"""
You should express what you want a model to do by \
providing instructions that are clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.


**Tactic2: Ask for a structured ouput**


*   JSON, HTML



In [17]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret Life of Bees",
    "author": "Lila Rose",
    "genre": "Romance"
  }
]


**Tactic 3: Ask the model to check whether conditions are satisfied**

In [18]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can ad some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!


**Tactic 4: "Few-shot" prompting**

In [19]:
prompt = f"""
Your task is to answer in a consistent sytle.

<child>: teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony origitantes from a single note; \
the most intrincate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree needs strong roots to weather a storm, we need a strong foundation of inner strength and determination to overcome life's challenges.


###Principles 2: Give the model time to "think"

**Tactic 1: Specify the steps required to complete task**

In [20]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck-Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line brakes.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Siblings Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes as Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home slightly battered but with their adventurous spirits undimmed.
2 - Les frères et sœurs Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais la malchance frappe lorsque Jack trébuche sur une pierre et dévale la colline, suivi de Jill, mais ils rentrent chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.
3 - Jack, Jill.
4 - {
     "french_summary": "Les frères et sœurs Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais la malchance frappe lorsque Jack trébuche sur une pierre et dévale la colline, suivi de Jill, mais ils rentrent chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.",
     "num_names": 2
   }


**ASk for output in a specific format**

In [21]:
prompt_2 = f"""
You task is to perform the following actions:
1- Summarize the following text delimited by 
 <> with 1 sentence.
 2 - Translate the summary into French.
 3 - List each name in the French summary.
 4 - Output a json object that contains the 
  follwing keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <List of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompetion for prompt 2:")
print(response)


Competion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down the hill, returning home slightly battered but still adventurous. 
Translation: Jack et Jill partent chercher de l'eau mais tombent et rentrent chez eux légèrement blessés mais toujours aventureux.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill partent chercher de l'eau mais tombent et rentrent chez eux légèrement blessés mais toujours aventureux.", "num_names": 2}


**Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion***

In [22]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x+ 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


**Note that the student's solution is actually not correct.**

**We can fix this by instructing the model to work out its own solution first.**

In [25]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
to solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```
Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's Solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x+ 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Steps to work out the solution and your solution here:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x+ 100,000 + 10x = 360x + 100,000

My solution is the same as the actual solution.

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


##Model Limitations: Hallucinations


*   Boie is a real company, the product name is not real.



In [26]:
prompt = f"""
Tell me about AeroGlide UltraSlim Tootbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Toothbrush by Boie is a unique toothbrush that is designed to provide a comfortable and effective brushing experience. It features a slim and lightweight design that makes it easy to hold and maneuver, and its bristles are made from a durable and flexible material that is gentle on teeth and gums.

The toothbrush also features a unique bristle pattern that helps to remove plaque and debris from hard-to-reach areas, and its compact size makes it ideal for travel or on-the-go use. Additionally, the AeroGlide UltraSlim Toothbrush is eco-friendly, as it is made from a recyclable material that is free from harmful chemicals and toxins.

Overall, the AeroGlide UltraSlim Toothbrush by Boie is a great choice for anyone looking for a high-quality, comfortable, and eco-friendly toothbrush that provides effective cleaning and a great brushing experience.


**Notes on using the OpenAI API outside of this classroom**
To install the Open AI library:

!pip install openai

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys).

You can either set it as the OPENAI_API_KEY environment variable before using the library:

!export OPENAI_API_KEY='sk-...'

Or, set openai.api_key to its value:

import openai
openai.api_key = "sk-..."

**A note about the backslash**


*   In the course, we are using a backslash \ to make the text fit on the screen without inserting newline '\n' characters.
*   GPT-3 isn't really affected whether you insert newline characters or not. But when working with LLMs in general, you may consider newline characters in your prompt may affect the model's performance.

